### Tune correction

This example shows how to run tune correction using the simulator or the virtual accelerator.
If you want to test the virtual accelerator you need to start the container before running the script.

In [1]:
import time

import numpy as np

from pyaml.accelerator import Accelerator
from pyaml.common.constants import ACTION_RESTORE
from pyaml.magnet.magnet import Magnet

In [ ]:
# ----- Load the configuration -----
# Remember to change the prefix for the live mode to the one matching
# your virtual accelerator before loading.

sr = Accelerator.load("BESSY2Tune.yaml")

28 Jan% 2026, 14:16:39 | WARNING | PyAML OA control system binding (0.1.1) initialized with name 'live' and prefix='a3744:'


In [3]:
# ----- Define a callback -----
# This callback is used to print output during the tune response measurement.


def tune_callback(step: int, action: int, m: Magnet, dtune: np.array):
    if action == ACTION_RESTORE:
        # On action restore, the measured delta tune is passed as argument
        print(f"Tune response: #{step} {m.get_name()} {dtune}")
    return True

In [4]:
# ----- Measure the tune response matrix-----
# You can measure the tune response matrix on either the design or live mode.

# sr.design.get_lattice().disable_6d()
tune_adjust = sr.design.tune
tune_adjust.response.measure(callback=tune_callback)
tune_adjust.response.save_json("tune-response.json")

Tune response: #0 Q3M2D1R [ 0.2948122  -1.59761611]
Tune response: #1 Q3M1T1R [ 0.31455894 -1.62602422]
Tune response: #2 Q3M2T1R [ 0.39697699 -1.488687  ]
Tune response: #3 Q3M1D2R [ 0.20694739 -1.66483728]
Tune response: #4 Q3M2D2R [ 0.2012383 -1.5957427]
Tune response: #5 Q3M1T2R [ 0.41065785 -1.45769981]
Tune response: #6 Q3M2T2R [ 0.37243911 -1.47809081]
Tune response: #7 Q3M1D3R [ 0.17999635 -1.57290023]
Tune response: #8 Q3M2D3R [ 0.16824072 -1.69184086]
Tune response: #9 Q3M1T3R [ 0.40143766 -1.53697391]
Tune response: #10 Q3M2T3R [ 0.42532148 -1.48886172]
Tune response: #11 Q3M1D4R [ 0.17907349 -1.74931195]
Tune response: #12 Q3M2D4R [ 0.18700724 -1.51871493]
Tune response: #13 Q3M1T4R [ 0.40510134 -1.37702393]
Tune response: #14 Q3M2T4R [ 0.39620807 -1.34034338]
Tune response: #15 Q3M1D5R [ 0.1820804  -1.56154372]
Tune response: #16 Q3M2D5R [ 0.18853823 -1.6711882 ]
Tune response: #17 Q3M1T5R [ 0.38067984 -1.39262069]
Tune response: #18 Q3M2T5R [ 0.38563368 -1.53580491]
Tune 

In [ ]:
# ----- Load the response matrix -----
# The example does the correction for the live mode
# but it can also be done on the design mode.

sr.design.tune.response.load_json("tune-response.json")

In [ ]:
# ----- Correct the tune -----

print("\nRun tune correction:")
print(
    f"Initial tunes: {
        np.array2string(sr.design.tune.readback(), precision=6, floatmode='fixed')
    }"
)
sr.design.tune.set([0.83, 0.84], iter=2, wait_time=3)
time.sleep(3)
print(
    f"Final tunes: {
        np.array2string(sr.design.tune.readback(), precision=6, floatmode='fixed')
    }"
)


Run tune correction:
Initial tunes: [0.848070 0.726778]
Final tunes: [0.830001 0.839958]
